# Export Network as Touchstone File

Frequently a `Network` is required to be saved as a [Touchstone file](https://en.wikipedia.org/wiki/Touchstone_file) so that it can be imported into another program such as [Keysight ADS](https://www.keysight.com/zz/en/products/software/pathwave-design-software/pathwave-advanced-design-system.html) or [Microwave Office](https://www.cadence.com/en_US/home/tools/system-analysis/rf-microwave-design/awr-design-environment-platform.html). The `Network.write_touchstone()` method allows `Network` objects to be exported in the appropriate format.

In [5]:
import skrf as rf
import numpy as np
from pathlib import Path

In this example, we are going to first generate a random 2-port `Network` and export the data as a Touchstone file using all the default arguments

In [6]:
# Handle for random generator
rand_gen = np.random.default_rng()

# Set the number of points
ports = 2

# Generate the frequency object
points = 11
freq = rf.Frequency(1, 50, points, 'GHz')

# Create random data to fill the Network
s_random = rand_gen.uniform(-1, 1, (freq.npoints, ports, ports)) + \
        1j * rand_gen.uniform(-1, 1, (freq.npoints, ports, ports))

# Generate the Network object
ntwk = rf.Network(s=s_random, frequency=freq, name='test_ntwk')
ntwk.comments = 'Default output'

# Define the save location
filepath = Path(f'test_network.s{ports}p')

# Save the Network as a touchstone file using the default arguments
print(ntwk.write_touchstone(filepath, return_string=True))

!Default output
! Created with skrf (http://scikit-rf.org).
# GHz S RI R 50.0 
!freq ReS11 ImS11 ReS21 ImS21 ReS12 ImS12 ReS22 ImS22
1.0 -0.464865151852476 0.4502139349896557 -0.946087157816933 -0.774157144858624 0.22978320604401903 -0.015382277631633245 -0.5143243821327415 0.5414888218758449
5.9 -0.7714731941571016 0.5035362767937888 -0.13676846127948927 -0.28407348971702495 0.22574007371745886 0.7585739185331353 -0.10651147209906475 0.8017532911041758
10.8 0.9283594792475798 -0.949741920331475 0.0013830072200151644 0.5869045175861258 -0.7303982230754289 0.4182901609474474 0.5005946980290608 -0.7192362826596026
15.7 0.05196345220484089 0.9250542904689592 -0.9756169835079893 -0.6532005774023946 0.29678736213869095 -0.2743856183602864 0.2781155976701659 0.36062628422326837
20.6 -0.07332329228960388 0.6134822411048317 -0.5177662945200454 0.41284206902464593 -0.2849256858620177 -0.5010232817863058 0.10007939815281519 -0.7011091912436347
25.5 0.30846454726129147 -0.2866491167667229 -0.8623

In this example, we are going to export the same 2-port `Network` as a Touchstone file, but use the formatting strings to format the columns of the resulting file.

In [7]:
# Get a copy of the network
ntwk_formatted = ntwk.copy()
ntwk_formatted.comments = 'Formatted output'

# Define the save location
formatted_filepath = Path(f'test_network_formatted.s{ports}p')

# Save the Network as a touchstone file using the string formatting options
freq_fmt = '{:<10.5f}'
data_fmt = '\t{:>10.5f}'
print(
    ntwk_formatted.write_touchstone(
        formatted_filepath,
        form='db',
        return_string=True,
        format_spec_freq=freq_fmt,
        format_spec_A=data_fmt,
        format_spec_B=data_fmt
    )
)

!Formatted output
! Created with skrf (http://scikit-rf.org).
# GHz S DB R 50.0 
!freq dBS11 angS11 dBS21 angS21 dBS12 angS12 dBS22 angS22
1.00000    	  -3.78001 	 135.91728 	   1.74467 	-140.70748 	 -12.75422 	  -3.82981 	  -2.53568 	 133.52619
5.90000    	  -0.71236 	 146.86774 	 -10.02599 	-115.70870 	  -2.03153 	  73.42779 	  -1.84321 	  97.56733
10.80000   	   2.46464 	 -45.65229 	  -4.62863 	  89.86499 	  -1.49692 	 150.20076 	  -1.14698 	 -55.16175
15.70000   	  -0.66297 	  86.78488 	   1.39407 	-146.19668 	  -7.86827 	 -42.75398 	  -6.83192 	  52.36053
20.60000   	  -4.18236 	  96.81565 	  -3.58010 	 141.43282 	  -4.78591 	-119.62635 	  -2.99668 	 -81.87623
25.50000   	  -7.51247 	 -42.90062 	   0.64358 	 143.20185 	  -5.19538 	-141.50957 	  -6.65092 	  32.15143
30.40000   	   0.15134 	  26.00150 	  -5.39670 	  24.70160 	  -0.56410 	  69.76471 	  -0.02690 	-107.76177
35.30000   	  -4.24885 	 113.33311 	  -5.29913 	-118.06431 	  -2.36891 	 136.29304 	  -9.06483 	 -50.96184
40.20

In this example, we are going to export a new 2-port `Network` as a Touchstone file, but we're going to include random noise data and format those output columns as well.

In [8]:
# Create random data to fill the Network
s_random = rand_gen.uniform(-1, 1, (freq.npoints, ports, ports)) + \
        1j * rand_gen.uniform(-1, 1, (freq.npoints, ports, ports))

# Create random noise data to fill the Network
nfmin_db_random = rand_gen.uniform(0, 10, freq.npoints)
g_opt_random = rand_gen.uniform(-1, 1, freq.npoints) + \
        1j * rand_gen.uniform(-1, 1, freq.npoints)
rn_random = rand_gen.uniform(size=freq.npoints)

# Generate the Network object
noisy_ntwk = rf.Network(s=s_random, frequency=freq, name='test_noisy_ntwk')
noisy_ntwk.comments = 'Formatted output with noise data'

# Add the noise data
noisy_ntwk.set_noise_a(noise_freq=freq, nfmin_db=nfmin_db_random, gamma_opt=g_opt_random, rn=rn_random)

# Define the save location
formatted_filepath = Path(f'test_noisy_network_formatted.s{ports}p')

# Save the Network as a touchstone file using the string formatting options
print(
    noisy_ntwk.write_touchstone(
        formatted_filepath,
        form='db',
        return_string=True,
        format_spec_freq=freq_fmt,
        format_spec_A=data_fmt,
        format_spec_B=data_fmt,
        format_spec_nf_freq=freq_fmt,
        format_spec_nf_min=data_fmt,
        format_spec_g_opt_mag=data_fmt,
        format_spec_g_opt_phase=data_fmt,
        format_spec_rn=data_fmt
    )
)

!Formatted output with noise data
! Created with skrf (http://scikit-rf.org).
# GHz S DB R 50.0 
!freq dBS11 angS11 dBS21 angS21 dBS12 angS12 dBS22 angS22
1.00000    	  -0.91328 	-130.97542 	  -1.29884 	 130.51454 	  -4.26300 	-173.56091 	  -1.87356 	-113.17161
5.90000    	  -1.46165 	  34.91404 	  -5.12067 	  43.04405 	  -0.70823 	-141.59246 	  -2.14551 	  47.87232
10.80000   	  -7.68630 	  31.44550 	   1.37477 	 -49.54463 	 -39.56968 	 -16.58557 	  -5.65258 	  -8.49307
15.70000   	  -0.90550 	-136.21873 	  -1.18593 	  15.14162 	  -1.69316 	-161.16082 	 -23.72235 	-165.91385
20.60000   	  -9.51858 	  89.43189 	  -4.74431 	  25.90892 	   1.38186 	 140.40619 	  -3.82659 	   6.48215
25.50000   	  -0.03824 	 -91.89634 	   1.75464 	  39.54287 	  -2.64012 	-148.02961 	  -6.09412 	 177.52691
30.40000   	  -1.85579 	  11.21887 	   0.24374 	-156.97826 	   1.38966 	 134.87892 	  -4.55095 	-136.33491
35.30000   	 -18.78104 	  17.95128 	   1.71843 	 -54.35505 	 -12.79699 	-123.19957 	  -3.37591 	

Finally, the Touchstone files can be imported into ADS or Microwave Office